In [ ]:
import seaborn as sns
import pandas as pd
from matplotlib import pyplot as plt




In [ ]:

# 1. Load Data
df_songs = pd.read_excel('Song_Data_With_Genres.xlsx')
df_climate = pd.read_csv('Global_Climate_Data_Formatted.csv', delimiter=';')

# 2. Data Cleaning
# Climate
df_climate['Average_Temp_Celsius'] = df_climate['Average_Temp_Celsius'].astype(str).str.replace(',', '.').astype(float)
df_climate['Rainfall_mm_per_year'] = pd.to_numeric(df_climate['Rainfall_mm_per_year'], errors='coerce')

# Songs - Country Mapping
country_corrections = {
    'United Arab': 'United Arab Emirates',
    'South Korea': 'South Korea', 
    'Ivory Coast': "Cote d'Ivoire",
    'Costa': 'Costa Rica',
    'Saudi': 'Saudi Arabia',
    'Türkiye': 'Turkey',
    'Czechia': 'Czech Republic'
}
df_songs['Country'] = df_songs['Country'].replace(country_corrections)

# 3. EDA Visualizations
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Exploratory Data Analysis (EDA)', fontsize=20)

# Plot 1: Climate - Temperature Distribution
sns.histplot(df_climate['Average_Temp_Celsius'], bins=20, kde=True, ax=axes[0, 0], color='orange')
axes[0, 0].set_title('Distribution of Average Temperature (°C)')
axes[0, 0].set_xlabel('Temperature')

# Plot 2: Climate - Rainfall Distribution
sns.histplot(df_climate['Rainfall_mm_per_year'], bins=20, kde=True, ax=axes[0, 1], color='blue')
axes[0, 1].set_title('Distribution of Annual Rainfall (mm)')
axes[0, 1].set_xlabel('Rainfall (mm)')

# Prepare Song Data (Explode Genres/Moods for global counts)
def split_and_explode(df, col):
    expanded = df.assign(**{col: df[col].str.split('/')}).explode(col)
    expanded[col] = expanded[col].str.strip()
    return expanded

df_genres = split_and_explode(df_songs, 'Genre')
df_moods = split_and_explode(df_songs, 'Mood')

# Plot 3: Top 15 Genres
top_genres = df_genres['Genre'].value_counts().head(15)
sns.barplot(x=top_genres.values, y=top_genres.index, ax=axes[1, 0], palette='viridis')
axes[1, 0].set_title('Top 15 Most Frequent Genres')
axes[1, 0].set_xlabel('Count')

# Plot 4: Top 15 Moods
top_moods = df_moods['Mood'].value_counts().head(15)
sns.barplot(x=top_moods.values, y=top_moods.index, ax=axes[1, 1], palette='magma')
axes[1, 1].set_title('Top 15 Most Frequent Moods')
axes[1, 1].set_xlabel('Count')

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.savefig('eda_summary_plots.png')

# 4. Summary Statistics to print
print("--- Data Summary ---")
print(f"Total Songs: {len(df_songs)}")
print(f"Total Countries in Song Data: {df_songs['Country'].nunique()}")
print(f"Total Countries in Climate Data: {len(df_climate)}")
print(f"Top 5 Genres:\n{top_genres.head(5)}")
print(f"Top 5 Moods:\n{top_moods.head(5)}")

# Check songs per country
songs_per_country = df_songs['Country'].value_counts()
print(f"\nSongs per Country stats:\nMin: {songs_per_country.min()}\nMax: {songs_per_country.max()}\nMode: {songs_per_country.mode()[0]}")

In [ ]:


# 1. Load Data
df_songs = pd.read_excel('Song_Data_With_Genres.xlsx')
df_climate = pd.read_csv('Global_Climate_Data_Formatted.csv', delimiter=';')

# 2. Data Cleaning
# Climate
df_climate['Average_Temp_Celsius'] = df_climate['Average_Temp_Celsius'].astype(str).str.replace(',', '.').astype(float)
df_climate['Rainfall_mm_per_year'] = pd.to_numeric(df_climate['Rainfall_mm_per_year'], errors='coerce')

# Songs - Country Mapping
country_corrections = {
    'United Arab': 'United Arab Emirates',
    'South Korea': 'South Korea',
    'Ivory Coast': "Cote d'Ivoire",
    'Costa': 'Costa Rica',
    'Saudi': 'Saudi Arabia',
    'Türkiye': 'Turkey',
    'Czechia': 'Czech Republic'
}
df_songs['Country'] = df_songs['Country'].replace(country_corrections)

# Merge
df_merged = pd.merge(df_songs, df_climate, on='Country', how='inner')

# 3. Explode & Aggregate
def get_proportions(df, col, song_counts):
    expanded = df.assign(**{col: df[col].str.split('/')}).explode(col)
    expanded[col] = expanded[col].str.strip()
    # Count per country
    counts = expanded.groupby(['Country', col]).size().unstack(fill_value=0)
    # Divide by total songs per country to get proportion
    return counts.div(song_counts, axis=0)

song_counts = df_merged.groupby('Country').size()
genre_props = get_proportions(df_merged, 'Genre', song_counts)
mood_props = get_proportions(df_merged, 'Mood', song_counts)

# 4. Filter for Top Genres/Moods (to avoid noise from rare ones)
top_genres = genre_props.sum().sort_values(ascending=False).head(20).index
top_moods = mood_props.sum().sort_values(ascending=False).head(20).index

genre_analysis = genre_props[top_genres]
mood_analysis = mood_props[top_moods]

# Add Climate Data
climate_cols = df_merged[['Country', 'Average_Temp_Celsius', 'Rainfall_mm_per_year']].drop_duplicates().set_index('Country')
genre_analysis = genre_analysis.join(climate_cols)
mood_analysis = mood_analysis.join(climate_cols)

# 5. Calculate Correlations
genre_corr = genre_analysis.corr()[['Average_Temp_Celsius', 'Rainfall_mm_per_year']].drop(['Average_Temp_Celsius', 'Rainfall_mm_per_year'])
mood_corr = mood_analysis.corr()[['Average_Temp_Celsius', 'Rainfall_mm_per_year']].drop(['Average_Temp_Celsius', 'Rainfall_mm_per_year'])

# Sort by Temperature correlation for better visualization
genre_corr = genre_corr.sort_values('Average_Temp_Celsius', ascending=False)
mood_corr = mood_corr.sort_values('Average_Temp_Celsius', ascending=False)

# 6. Visualization
fig, axes = plt.subplots(1, 2, figsize=(15, 10))

sns.heatmap(genre_corr, annot=True, cmap='RdBu_r', center=0, ax=axes[0], vmin=-0.6, vmax=0.6)
axes[0].set_title('Correlation: Genre vs Climate')

sns.heatmap(mood_corr, annot=True, cmap='RdBu_r', center=0, ax=axes[1], vmin=-0.6, vmax=0.6)
axes[1].set_title('Correlation: Mood vs Climate')

plt.tight_layout()
plt.savefig('correlation_heatmap.png')

# Print top correlations for user
print("Top Positive Correlations (Genre vs Temp):")
print(genre_corr['Average_Temp_Celsius'].head(3))
print("\nTop Negative Correlations (Genre vs Temp):")
print(genre_corr['Average_Temp_Celsius'].tail(3))

print("\nTop Positive Correlations (Mood vs Temp):")
print(mood_corr['Average_Temp_Celsius'].head(3))
print("\nTop Negative Correlations (Mood vs Temp):")
print(mood_corr['Average_Temp_Celsius'].tail(3))

In [ ]:


# 1. Load & Clean
df_songs = pd.read_excel('Song_Data_With_Genres.xlsx')
df_climate = pd.read_csv('Global_Climate_Data_Formatted.csv', delimiter=';')

# Climate Clean
df_climate['Average_Temp_Celsius'] = df_climate['Average_Temp_Celsius'].astype(str).str.replace(',', '.').astype(float)
df_climate['Rainfall_mm_per_year'] = pd.to_numeric(df_climate['Rainfall_mm_per_year'], errors='coerce')

# Country Clean
country_corrections = {
    'United Arab': 'United Arab Emirates',
    'South Korea': 'South Korea',
    'Ivory Coast': "Cote d'Ivoire",
    'Costa': 'Costa Rica',
    'Saudi': 'Saudi Arabia',
    'Türkiye': 'Turkey',
    'Czechia': 'Czech Republic'
}
df_songs['Country'] = df_songs['Country'].replace(country_corrections)

# Merge
df_merged = pd.merge(df_songs, df_climate, on='Country', how='inner')

# Helper for exploding
def split_and_explode(df, col):
    expanded = df.assign(**{col: df[col].str.split('/')}).explode(col)
    expanded[col] = expanded[col].str.strip()
    return expanded

# Explode Genres for analysis
df_genres_exploded = split_and_explode(df_merged, 'Genre')
df_moods_exploded = split_and_explode(df_merged, 'Mood')

# --- VISUALIZATION GENERATION ---
fig = plt.figure(figsize=(20, 18))
plt.suptitle('Advanced Climate-Music Analysis', fontsize=24)

# 1. Box Plot: Temperature Ranges of Top Genres
# Filter for top 12 genres to keep it readable
top_12_genres = df_genres_exploded['Genre'].value_counts().head(12).index
df_box = df_genres_exploded[df_genres_exploded['Genre'].isin(top_12_genres)]

# Sort genres by median temperature for the plot
order = df_box.groupby('Genre')['Average_Temp_Celsius'].median().sort_values(ascending=False).index

ax1 = plt.subplot(2, 2, 1)
sns.boxplot(x='Average_Temp_Celsius', y='Genre', data=df_box, order=order, palette='coolwarm', ax=ax1)
ax1.set_title('Temperature Range of Top Genres\n(Ordered from Hot to Cold Preference)', fontsize=16)
ax1.set_xlabel('Average Country Temperature (°C)')

# 2. Comparative Bar Chart: Hot vs Cold Countries (Genre Preference)
# Define Zones
df_merged_unique = df_merged[['Country', 'Average_Temp_Celsius']].drop_duplicates()
cold_thresh = df_merged_unique['Average_Temp_Celsius'].quantile(0.33)
hot_thresh = df_merged_unique['Average_Temp_Celsius'].quantile(0.67)

def get_zone(temp):
    if temp < cold_thresh: return 'Cold Zone'
    elif temp > hot_thresh: return 'Hot Zone'
    else: return 'Temperate Zone'

df_genres_exploded['Climate_Zone'] = df_genres_exploded['Average_Temp_Celsius'].apply(get_zone)

# Calculate % of genres within each zone
zone_counts = df_genres_exploded.groupby(['Climate_Zone', 'Genre']).size().reset_index(name='Count')
total_per_zone = df_genres_exploded.groupby('Climate_Zone')['Genre'].count().reset_index(name='Total')
zone_counts = pd.merge(zone_counts, total_per_zone, on='Climate_Zone')
zone_counts['Percentage'] = (zone_counts['Count'] / zone_counts['Total']) * 100

# Filter for plotting: Take top 5 genres from Hot and Top 5 from Cold
top_hot = zone_counts[zone_counts['Climate_Zone']=='Hot Zone'].nlargest(5, 'Percentage')['Genre'].tolist()
top_cold = zone_counts[zone_counts['Climate_Zone']=='Cold Zone'].nlargest(5, 'Percentage')['Genre'].tolist()
combined_top = list(set(top_hot + top_cold))

df_bar = zone_counts[zone_counts['Genre'].isin(combined_top) & (zone_counts['Climate_Zone'] != 'Temperate Zone')]

ax2 = plt.subplot(2, 2, 2)
sns.barplot(x='Percentage', y='Genre', hue='Climate_Zone', data=df_bar, palette={'Hot Zone': 'red', 'Cold Zone': 'blue'}, ax=ax2)
ax2.set_title('Genre Popularity: Hot vs. Cold Zones', fontsize=16)
ax2.set_xlabel('Percentage of Total Songs in Zone')

# 3. Mood Distribution by Climate Zone (Grouped Bar)
# We'll use a specific set of moods to reduce noise
target_moods = ['Party', 'Dance', 'Sad', 'Dark', 'Chill', 'Energetic', 'Romantic']
df_moods_exploded['Climate_Zone'] = df_moods_exploded['Average_Temp_Celsius'].apply(get_zone)
mood_zone_counts = df_moods_exploded[df_moods_exploded['Mood'].isin(target_moods)].groupby(['Climate_Zone', 'Mood']).size().reset_index(name='Count')

# Normalize
total_mood_per_zone = df_moods_exploded[df_moods_exploded['Mood'].isin(target_moods)].groupby('Climate_Zone')['Mood'].count().reset_index(name='Total')
mood_zone_counts = pd.merge(mood_zone_counts, total_mood_per_zone, on='Climate_Zone')
mood_zone_counts['Percentage'] = (mood_zone_counts['Count'] / mood_zone_counts['Total']) * 100

ax3 = plt.subplot(2, 2, 3)
sns.barplot(x='Mood', y='Percentage', hue='Climate_Zone', data=mood_zone_counts[mood_zone_counts['Climate_Zone'] != 'Temperate Zone'], 
            palette={'Hot Zone': 'red', 'Cold Zone': 'blue'}, ax=ax3)
ax3.set_title('Mood Preferences: Hot vs. Cold Zones', fontsize=16)
ax3.set_ylabel('Relative Frequency (%)')

# 4. Scatter Plot: Countries clustered by Dominant Genre
# Find dominant genre per country
country_dominant_genre = df_genres_exploded.groupby(['Country', 'Genre']).size().reset_index(name='Count')
country_dominant_genre = country_dominant_genre.sort_values(['Country', 'Count'], ascending=[True, False])
country_dominant_genre = country_dominant_genre.groupby('Country').first().reset_index()

# Merge back climate
country_scatter = pd.merge(country_dominant_genre, df_climate[['Country', 'Average_Temp_Celsius', 'Rainfall_mm_per_year']], on='Country')

# Filter for the most common dominant genres to avoid too many colors
top_dom_genres = country_scatter['Genre'].value_counts().head(6).index
country_scatter['Genre_Label'] = country_scatter['Genre'].apply(lambda x: x if x in top_dom_genres else 'Other')

ax4 = plt.subplot(2, 2, 4)
sns.scatterplot(x='Average_Temp_Celsius', y='Rainfall_mm_per_year', hue='Genre_Label', style='Genre_Label', s=100, data=country_scatter, ax=ax4, palette='tab10')
ax4.set_title('Countries Clustered by Dominant Genre', fontsize=16)
ax4.set_xlabel('Average Temp (°C)')
ax4.set_ylabel('Annual Rainfall (mm)')
ax4.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()
plt.savefig('advanced_visualizations.png')